In [39]:
import pandas as pd
dataset_spotify = pd.read_csv('apiData.csv', header=0)
dataset_weather_alexandria = pd.read_csv('alexandria.csv', header=0)
dataset_weather = dataset_weather_alexandria[['date', 'precipcover', 'conditions']]

In [40]:
merged_dataset = dataset_spotify.merge(dataset_weather, on='date',)
#merged_dataset.to_csv('./cleaned_data/merged.csv', index=False)

In [41]:
def binary_precipitation(value):
    if value != 0:
        return 1
    else:
        return 0


Convert all raining values into a binary, "Is raining" or not

In [42]:
merged_dataset['precipcover'] = merged_dataset['precipcover'].apply(binary_precipitation)
display(merged_dataset)

,Year,Month,Day,time,trackName,albumName,artistName,playlists.name,track_id,date,...,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,precipcover,conditions
0,2023,6,22,02:00:00,"Amish Paradise (Parody of ""Gangsta's Paradise""...",Bad Hair Day,"""Weird Al"" Yankovic",HOmeWorK BANgers,5r96TaQquRrlo3Ym3ZlSL2,6/22/2023,...,0.1720,0.103000,0.000000,0.2670,0.4830,80.902,202920,4,0,Partially cloudy
1,2023,6,22,02:00:00,"Amish Paradise (Parody of ""Gangsta's Paradise""...",Bad Hair Day,"""Weird Al"" Yankovic",HOmeWorK BANgers,5r96TaQquRrlo3Ym3ZlSL2,6/22/2023,...,0.1720,0.103000,0.000000,0.2670,0.4830,80.902,202920,4,1,"Rain, Overcast"
2,2023,6,22,20:03:00,I'm Born To Run,What We Live For,American Authors,The Knew,6zDs6zI94L761vd0cVScTT,6/22/2023,...,0.0486,0.066200,0.000000,0.0788,0.6750,126.041,206920,4,0,Partially cloudy
3,2023,6,22,20:03:00,I'm Born To Run,What We Live For,American Authors,The Knew,6zDs6zI94L761vd0cVScTT,6/22/2023,...,0.0486,0.066200,0.000000,0.0788,0.6750,126.041,206920,4,1,"Rain, Overcast"
4,2023,6,22,10:09:00,Do I Wanna Know?,AM,Arctic Monkeys,Me and michael,5FVd6KXrgO9B3JPmC8OPst,6/22/2023,...,0.0323,0.186000,0.000263,0.2170,0.4050,85.030,272394,4,0,Partially cloudy
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1989,2023,6,19,17:29:00,Welcome to the Black Parade,The Black Parade,My Chemical Romance,Missing Her,5wQnmLuC1W7ATsArWACrgW,6/19/2023,...,0.0752,0.000289,0.000110,0.2220,0.2360,96.950,311107,4,1,"Rain, Partially cloudy"
1990,2023,7,30,13:31:00,So Tied Up,LA DIVINE,Cold War Kids,Missing Her,3msS7xaelR4Oa0n7IwHkgF,7/30/2023,...,0.0402,0.030900,0.000022,0.1790,0.8100,104.993,189507,4,0,Clear
1991,2023,7,30,13:31:00,So Tied Up,LA DIVINE,Cold War Kids,Missing Her,3msS7xaelR4Oa0n7IwHkgF,7/30/2023,...,0.0402,0.030900,0.000022,0.1790,0.8100,104.993,189507,4,0,Partially cloudy
1992,2023,7,30,05:30:00,Les pensées,Foundations,The Architect,Missing Her,0fira69E5qwYJNSvzEOCs7,7/30/2023,...,0.0392,0.015100,0.020500,0.0969,0.0392,97.975,238156,4,0,Clear


In [43]:
columns_to_drop = ['Year', 'Month', 'Day', 'time', 'trackName', 'albumName', 'artistName',
                   'playlists.name', 'track_id', 'hour', 'date', 'precipcover', 'conditions']

x = merged_dataset.drop(columns=columns_to_drop, axis=1)

x.fillna(0, inplace=True)
#x.to_csv('./cleaned_data/cleaned.csv', index=False)
y = merged_dataset['precipcover']

display(x)

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,0.728,0.448,8,-10.540,1,0.1720,0.103000,0.000000,0.2670,0.4830,80.902,202920,4
1,0.728,0.448,8,-10.540,1,0.1720,0.103000,0.000000,0.2670,0.4830,80.902,202920,4
2,0.620,0.930,8,-5.615,1,0.0486,0.066200,0.000000,0.0788,0.6750,126.041,206920,4
3,0.620,0.930,8,-5.615,1,0.0486,0.066200,0.000000,0.0788,0.6750,126.041,206920,4
4,0.548,0.532,5,-7.596,1,0.0323,0.186000,0.000263,0.2170,0.4050,85.030,272394,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1989,0.217,0.905,2,-4.103,1,0.0752,0.000289,0.000110,0.2220,0.2360,96.950,311107,4
1990,0.587,0.894,0,-4.184,0,0.0402,0.030900,0.000022,0.1790,0.8100,104.993,189507,4
1991,0.587,0.894,0,-4.184,0,0.0402,0.030900,0.000022,0.1790,0.8100,104.993,189507,4
1992,0.678,0.535,10,-8.592,0,0.0392,0.015100,0.020500,0.0969,0.0392,97.975,238156,4


In [44]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

Using a decision tree classifier, I am able to predict if it is raining based on the type of music I am listening to.

In [45]:
from sklearn.tree import DecisionTreeClassifier
tree = DecisionTreeClassifier()
tree = tree.fit(x_train, y_train)
y_pred = tree.predict(x_test)

In [46]:
from sklearn import metrics
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.6466165413533834


In [47]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [48]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(x_train, y_train)
y_pred = knn.predict(x_test)

In [49]:
accuracy = metrics.accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.6516290726817042
